在这里就可以共享工作了~

【一名路过的小学生】

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import warnings
import tqdm
warnings.filterwarnings("ignore")

matplotlib.rcParams['font.sans-serif']=['SimHei']   # 用黑体显示中文
matplotlib.rcParams['axes.unicode_minus']=False     # 正常显示负号
    
    
df1 = pd.read_csv("../input/airbnbeasy/listings.csv")
# df1.info()
df2 = pd.read_csv("../input/beijing-airbnb-first/listings.csv")
df2 = df2.loc[:,['id','amenities','review_scores_rating']]
df = pd.merge(df1,df2,how = 'left')
df = df.drop('neighbourhood_group',axis=1)
df = df[-df['name'].isnull()]
df = df[-df['host_name'].isnull()]
# df.info()
# df.describe()

# 用均方差法找出价格异常值,标准差能反映一个数据集的离散程度
s = df['price']
# 序列中的价格和平均价格的差距
zscore = s - s.mean()
# 这里用3.5倍sigma
df['isOutlier'] = zscore.abs() > 3.5 * s.std()
# 输出异常的样本
df_out = df[df['isOutlier'] == True]
# print(df_out)
df = df[-df['isOutlier'] == True]
df = df.drop('isOutlier',axis = 1)

ame_list = df['amenities'].tolist()
for i in range(df.shape[0]):
    ame_list[i] = list(ame_list[i].replace('"',"").replace('[',"").replace(']','').replace(' ','').split(','))
amenities = {}

#选取top5
"""
for i in range(df.shape[0]):
    for amenity in ame_list[i]:
        if amenity in amenities:
            amenities[amenity] += 1
        else:
            amenities[amenity] = 1
"""
    
# print(sorted(amenities.items(),key=lambda x:x[1],reverse = True))

In [ ]:
#获取前五的设备
ame_topfive = ['Longtermstaysallowed','Wifi','Airconditioning','Shampoo','Heating']
for ame_ in ame_topfive:
    df[ame_] = np.zeros([df.shape[0],1],dtype = 'int')

#数据预处理，大概需要1min
for i in tqdm.tqdm(range(df.shape[0])):
    for ame_ in ame_topfive:
         if (ame_ in ame_list[i]):
             df[ame_].iloc[i] = 1
         else:
             df[ame_].iloc[i] = 0
            
df.info()    

In [ ]:
import numpy as np 
from scipy import stats

#author: CLS

# REF: https://zhuanlan.zhihu.com/p/128905132

#复制一份data用于数据处理
data = df.copy()
data["price_level"] = np.zeros([data.shape[0],1],dtype = 'int')

#根据价格分级
# REF: https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.21.72e8b89crRCH4F&postId=125694
def price_level(price):
    if price<=100:
        return 1
    elif price<=500 :
        return 2
    elif price<=1000:
        return 3
    elif price<=3000:
        return 4
    elif price<=5000:
        return 5
    elif price<=10000:
        return 6
    else:
        return 7
    
#价格分级处理
for i in tqdm.tqdm(range(df.shape[0])):
    data["price_level"].iloc[i] = price_level(data["price"].iloc[i])
data.info()

In [ ]:
def cross_stas(attr):
    #使用卡方检验检测相关性，备选假设为无关，得到p值越小表示因素无关的可能性越小
    ctb = pd.crosstab(data['price_level'],data[attr],margins=True,margins_name='sum')
    #print(ctb)
    observed = ctb[[0,1]]
    #print(observed)
    chi2,p,dof,expected = stats.chi2_contingency(observed=observed)
    return p

ames = ['Longtermstaysallowed','Wifi','Airconditioning','Shampoo','Heating']
pvalues = []
for ame in ames:
    pvalues.append(cross_stas(ame))
print(ame_topfive)
print(pvalues)

#输出pvalue
#如果显著性水平设置为0.01，可以看出pvalue都很小，也即设备和价格等级相关性很大
# [7.958410322246654e-08, 5.954281058563225e-13, 6.947555929043191e-22, 1.2508017034655443e-123, 9.04569067521791e-07]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

def level_price(level):
    if level==1:
        return '0-100'
    elif level==2 :
        return "100-500"
    elif level==3:
        return "500-1000"
    elif level==4:
        return "1000-3000"
    elif level==5:
        return "3000-5000"
    elif level==6:
        return "5000-10000"
    elif level==7:
        return ">10000"

def KNN(attr):
    x_=data[attr]
    x_=x_.to_dict(orient="records")
    y_=data["price_level"]
    
    #训练集和测试集划分
    x_train,x_test,y_train,y_test=train_test_split(x_,y_)
    
    transfer_dic=DictVectorizer()
    x_train=transfer_dic.fit_transform(x_train)
    x_test=transfer_dic.transform(x_test)
    
    estimator=KNeighborsClassifier(n_neighbors=6)
    estimator.fit(x_train,y_train.values.ravel())
    
    score = estimator.score(x_test,y_test)
    return score

#返回score =（1-u/v）
# u=((y_true - y_pred) ** 2).sum()     v=((y_true - y_true.mean()) ** 2).sum()

ames = ['Longtermstaysallowed','Wifi','Airconditioning','Shampoo','Heating']
print(KNN(ames))
#只使用ame_topfive进行KNN，score仅有0.59

In [ ]:
locates = ["neighbourhood","latitude","longitude"]
print(KNN(locates))
#单纯使用位置信息KNN的score达到了0.63

In [ ]:
attrs = locates + ames + ['room_type']
print(attrs)
print(KNN(attrs))
#联合使用达到0.67
#可以看到，考虑的因素越多，KNN聚类的效果更好

In [ ]:
print(KNN(['room_type']))
#而仅仅使用room_type属性效果只有0.38

In [ ]:
# REF: https://blog.csdn.net/hubingshabi/article/details/80172608
# 尝试进行回归分析

from sklearn import linear_model        
model = linear_model.LinearRegression()
attr = ['Longtermstaysallowed','Wifi','Airconditioning','Shampoo','Heating']
x = data[attr]
y = data["price"]
model.fit(x, y)
print(model.score(x,y))
# 0.0004526977460965753

# 回归的效果很差
# 虽然相关性很强，但是并非线性相关，不适合做回归
# 用单因素回归效果也不好: Wifi: 9.070240428143972e-05

In [ ]:
from sklearn import linear_model        
model = linear_model.LinearRegression()
attr = ['Longtermstaysallowed','Wifi','Airconditioning','Shampoo','Heating']
x = data[attr]
y = data["price_level"]
model.fit(x, y)
print(model.score(x,y))
# 0.0004526977460965753

In [ ]:
from sklearn import linear_model 
model = linear_model.LogisticRegression()
attr = ['Longtermstaysallowed','Wifi','Airconditioning','Shampoo','Heating']
x = data[attr]
y = data["price_level"]
model.fit(x, y)
print(model.score(x,y))
#逻辑斯特回归score达到0.58

In [ ]:
#决策树
import sklearn.tree 
def DT(attr):
    x_=data[attr]
    x_=x_.to_dict(orient="records")
    y_=data["price_level"]
    
    #训练集和测试集划分
    x_train,x_test,y_train,y_test=train_test_split(x_,y_)
    
    transfer_dic=DictVectorizer()
    x_train=transfer_dic.fit_transform(x_train)
    x_test=transfer_dic.transform(x_test)
    
    estimator=sklearn.tree.DecisionTreeRegressor(max_depth=10)
    estimator.fit(x_train,y_train.values.ravel())
    
    score = estimator.score(x_test,y_test)
    return score

attrs = locates + ames + ['room_type']
print(DT(attrs))
#仅有0.3的准确率，决策效果也不好

In [ ]:
import sklearn.ensemble

def RFR(attr):
    x_=data[attr]
    x_=x_.to_dict(orient="records")
    y_=data["price_level"]
    
    #训练集和测试集划分
    x_train,x_test,y_train,y_test=train_test_split(x_,y_)
    
    transfer_dic=DictVectorizer()
    x_train=transfer_dic.fit_transform(x_train)
    x_test=transfer_dic.transform(x_test)
    
    estimator = sklearn.ensemble.RandomForestRegressor(max_depth=10, n_estimators=10, min_samples_split=2)
    estimator.fit(x_train,y_train.values.ravel())
    
    score = estimator.score(x_test,y_test)
    return score
    
attrs = locates + ames + ['room_type']
print(RFR(attrs))
#采用10棵决策树做集成的随机森林也仅仅达到了0.4，说明该算法并不适合

In [ ]:
plt.hist(df['price'], bins=40, facecolor="blue", edgecolor="black", alpha=0.7)
# 显示横轴标签
plt.xlabel("Price")
# 显示纵轴标签
plt.ylabel("Frequency")
# 显示图标题
plt.title("Price Histogram")
plt.show()

plt.hist(df[df['price']<=1000]['price'], bins=40, facecolor="blue", edgecolor="black", alpha=0.7)
# 显示横轴标签
plt.xlabel("Price")
# 显示纵轴标签
plt.ylabel("Frequency")
# 显示图标题
plt.title("Price Histogram | Price < 1000")
plt.show()

In [ ]:
plt.hist(df['review_scores_rating'], bins=40, facecolor="blue", edgecolor="black", alpha=0.7)
# 显示横轴标签
plt.xlabel("Score")
# 显示纵轴标签
plt.ylabel("Frequency")
# 显示图标题
plt.title("Price Histogram")
plt.show()

plt.hist(df[df['review_scores_rating']>=80]['review_scores_rating'], bins=40, facecolor="blue", edgecolor="black", alpha=0.7)
# 显示横轴标签
plt.xlabel("Score")
# 显示纵轴标签
plt.ylabel("Frequency")
# 显示图标题
plt.title("Score Histogram | Score > 80")
plt.show()

In [ ]:
name_list = ['Shared room','Private room','Entire home/apt']
freq = [list(df['room_type']).count('Shared room'),list(df['room_type']).
        count('Private room'),list(df['room_type']).count('Entire home/apt')]
plt.barh(range(len(freq)), freq,tick_label = name_list)
plt.show()



In [ ]:
df_tmp = pd.DataFrame({'Shared room' : df['price'][df["price"] <= 1000][df['room_type'] =='Shared room'],
                       'Private room' : df['price'][df["price"] <= 1000][df['room_type']=='Private room'],
                       'Entire home/apt' : df['price'][df["price"] <= 1000][df['room_type']=='Entire home/apt'],
                      },columns = ['Shared room','Private room','Entire home/apt'])

df_tmp.plot.hist(stacked = True, alpha = 0.5)
df_tmp.plot.density(stacked = True, alpha=0.5)


In [ ]:
df_tmp1 = df.loc[:,['price','neighbourhood']]  
df_tmp1 = df_tmp1.groupby('neighbourhood').mean()


df_tmp = df_tmp1.sort_values('price',ascending = False)[:10]
print(df_tmp)
df_tmp.plot.barh()
# df_tmp.plot.pie(subplots = True)




In [ ]:
df_tmp2 = df.loc[:,['price','neighbourhood']]  
df_tmp2 = df_tmp2.groupby('neighbourhood').size().to_frame().reset_index()
df_tmp2.columns = ['neighbourhood','count']

df_tmp = df_tmp2.sort_values('count',ascending = False)[:10].reset_index().drop('index',axis=1)
print(df_tmp)
df_tmp.plot.barh(x='neighbourhood',y='count')


In [ ]:
df_tmp3 = pd.merge(df_tmp1,df_tmp2,on='neighbourhood')
print(df_tmp3)
df_tmp3.plot.bar(x='neighbourhood',stacked = False, alpha = 0.5)
#df_tmp3.plot.density(stacked = True, alpha=0.5)

In [ ]:
subsets=['price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
#箱型图，获取数据大致分布情况，查看离群点信息
fig,axes=plt.subplots(len(subsets),1,figsize=(20,10))
plt.subplots_adjust(hspace=1)
for i,subset in enumerate(subsets):
    sns.boxplot(df[subset],ax=axes[i],whis=2,orient='h')


In [ ]:
#探索性数据分析，数据集中性
def get_con(df):
    subsets=['price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
    data={}
    for i in subsets:
        data.setdefault(i,[])
        data[i].append(df[i].skew())
        data[i].append(df[i].kurt())
        data[i].append(df[i].mean())
        data[i].append(df[i].std())
        data[i].append(df[i].std()/df[i].mean())
        data[i].append(df[i].max()-df[i].min())
        data[i].append(df[i].quantile(0.75)-df[i].quantile(0.25))
        data[i].append(df[i].median())
    data_df=pd.DataFrame(data,index=['偏度','峰度','均值','标准差','变异系数','极差','四分位距','中位数'],columns=subsets)
    return data_df.T
df2=get_con(df)
df2

In [ ]:
import geopandas as gpd
from shapely.geometry import Point # 经纬度转换为点
import mapclassify
import matplotlib.pyplot as plt
%pylab inline
%matplotlib inline

nbhd_gpd = gpd.GeoDataFrame.from_file('../input/airbnbeasy/neighbourhoods.geojson') 
# nbhd_gpd.head(3)

def df_to_gpd(df, label_longitude, label_latitude):
    df["geometry"] = list(zip(df[label_longitude],df[label_latitude]))
    df["geometry"] = df["geometry"].apply(Point)
    df = gpd.GeoDataFrame(df)
    del df[label_latitude]
    del df[label_longitude]
    return df

listings_gpd = df_to_gpd(df, "longitude", "latitude")
# listings_gpd.info()
# listings_gpd.head(3)
#base = nbhd_gpd.plot(color='white', edgecolor='black',figsize=(15, 15))
#listings_gpd.plot(ax=base, color='red', marker="o", markersize=50, alpha=0.01) #在底图上叠加餐厅点数据
#plt.gca().xaxis.set_major_locator(plt.NullLocator())#去掉x轴刻度
#plt.gca().yaxis.set_major_locator(plt.NullLocator())#去掉y轴刻度

nbhd_groupby = listings_gpd.groupby("neighbourhood").size() # Series格式
nbhd_groupby = nbhd_groupby.to_frame().reset_index() # 转换为dataframe格式
nbhd_groupby.columns = ["neighbourhood", "count"] # 更改列名，方便操作
# nbhd_groupby.info()
# print(nbhd_gpd)
# print(nbhd_groupby)

nbhd_groupby_merge = pd.merge(nbhd_gpd, nbhd_groupby, on="neighbourhood", how="left")

nbhd_groupby_merge = nbhd_groupby_merge.drop("neighbourhood_group", axis=1)
nbhd_groupby_merge = nbhd_groupby_merge[-nbhd_groupby_merge["geometry"].isnull()]
print(nbhd_groupby_merge)
# nbhd_groupby_merge.head(3)

base = nbhd_groupby_merge.plot(column="count", cmap='Blues', scheme="boxplot"
                              , edgecolor='black',legend=True, figsize=(15, 15))
listings_gpd.plot(ax=base, color='purple', marker="o", markersize=50, alpha=0.02) #在底图上叠加房源点数据
plt.title("Airbnb Distribution", fontsize=30)
plt.gca().xaxis.set_major_locator(plt.NullLocator())#去掉x轴刻度
plt.gca().yaxis.set_major_locator(plt.NullLocator())#去掉y轴刻度